# Desafio Store Sales

## Carregamento dos dados

### Importação das bibliotecas necessárias à seção

In [1]:
import pandas as pd
import numpy as np
import pyarrow
from datetime import datetime, timedelta
import calendar
import multiprocessing 

pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True, precision=6)

### Importação dos datasets CSV e ajuste dos tipos das variáveis (histórico)

In [2]:
# Apenas registro histórico, caso seja necessário recuperar os arquivos do início
"""
treino = pd.read_csv("./csv/train.csv")
treino["date"] = pd.to_datetime(treino["date"])

teste = pd.read_csv("./csv/test.csv")
teste["date"] = pd.to_datetime(teste["date"])

feriados = pd.read_csv("./csv/holidays_events.csv")
feriados["date"] = pd.to_datetime(feriados["date"])

gasolina = pd.read_csv("./csv/oil.csv")
gasolina["date"] = pd.to_datetime(gasolina["date"])

transacoes = pd.read_csv("./csv/transactions.csv")
transacoes["date"] = pd.to_datetime(transacoes["date"])

lojas = pd.read_csv("s./csv/tores.csv") # não tem data envolvida. Dropa city/state?

# Transformação das demais variáveis categóricas efetivamente em categorias
treino["store_nbr"] = treino["store_nbr"].astype("category")
treino["family"] = treino["family"].astype("category")

teste["store_nbr"] = teste["store_nbr"].astype("category")
teste["family"] = teste["family"].astype("category")

feriados["type"] = feriados["type"].astype("category")
feriados["locale"] = feriados["locale"].astype("category")
feriados["locale_name"] = feriados["locale_name"].astype("category")
feriados["transferred"] = feriados["transferred"].astype("category")
feriados = feriados[feriados["transferred"] == False]
feriados.drop(feriados[feriados["date"] < "2013-01-01"].index, axis = 0, inplace = True)
feriados.drop(feriados[feriados["date"] >= "2017-09-01"].index, axis = 0, inplace = True)
feriados.reset_index(inplace = True)
feriados.drop("index", axis = 1, inplace = True)
feriados["f_nacional"] = 0
feriados["f_regional"] = 0
feriados["f_local"] = 0
    # Cria as dummies
for i in range(len(feriados)):
    if feriados.loc[i, "locale"] == "National":
        feriados.loc[i, "f_nacional"] = 1
    elif feriados.loc[i, "locale"] == "Regional":
        feriados.loc[i, "f_regional"] = 1
    elif feriados.loc[i, "locale"] == "Local":
        feriados.loc[i, "f_local"] = 1


gasolina.fillna(method='bfill', inplace=True)

transacoes["store_nbr"] = transacoes["store_nbr"].astype("category")

lojas["store_nbr"] = lojas["store_nbr"].astype("category")
lojas["type"] = lojas["type"].astype("category")
lojas["cluster"] = lojas["cluster"].astype("category")

# Exportação para parquet dos dados já brevemente ajustados 

teste.to_parquet("./parquet/teste.parquet")
treino.to_parquet("./parquet/treino.parquet")
feriados.to_parquet("./parquet/feriados.parquet")
gasolina.to_parquet("./parquet/gasolina.parquet")
transacoes.to_parquet("./parquet/transacoes.parquet")
lojas.to_parquet("./parquet/lojas.parquet")
"""

'\ntreino = pd.read_csv("./csv/train.csv")\ntreino["date"] = pd.to_datetime(treino["date"])\n\nteste = pd.read_csv("./csv/test.csv")\nteste["date"] = pd.to_datetime(teste["date"])\n\nferiados = pd.read_csv("./csv/holidays_events.csv")\nferiados["date"] = pd.to_datetime(feriados["date"])\n\ngasolina = pd.read_csv("./csv/oil.csv")\ngasolina["date"] = pd.to_datetime(gasolina["date"])\n\ntransacoes = pd.read_csv("./csv/transactions.csv")\ntransacoes["date"] = pd.to_datetime(transacoes["date"])\n\nlojas = pd.read_csv("s./csv/tores.csv") # não tem data envolvida. Dropa city/state?\n\n# Transformação das demais variáveis categóricas efetivamente em categorias\ntreino["store_nbr"] = treino["store_nbr"].astype("category")\ntreino["family"] = treino["family"].astype("category")\n\nteste["store_nbr"] = teste["store_nbr"].astype("category")\nteste["family"] = teste["family"].astype("category")\n\nferiados["type"] = feriados["type"].astype("category")\nferiados["locale"] = feriados["locale"].astype

### Feature Engeneering parcial (histórico)

In [3]:
# Apenas registro histórico, caso seja necessário recuperar os arquivos do início
"""
###
# Dataset de treino
### 

# store_type
for i in range(len(treino["store_type"])):
    treino["store_type"].iloc[i] = lojas["type"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# cluster
for i in range(len(treino["cluster"])):
    treino["cluster"].iloc[i] = lojas["cluster"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

    
# holidays_events
treino["f_nacional"] = 0
treino["f_regional"] = 0
treino["f_local"] = 0
for i in range(len(feriados)):
    if feriados.loc[i, "f_nacional"] == 1:
        mask = treino["date"] == feriados.loc[i, "date"]
        treino.loc[mask, "f_nacional"] = 1
    elif feriados.loc[i, "f_regional"] == 1:
        mask = treino["date"] == feriados.loc[i, "date"]
        treino.loc[mask, "f_regional"] = 1
    elif feriados.loc[i, "f_local"] == 1:
        mask = treino["date"] == feriados.loc[i, "date"]
        treino.loc[mask, "f_local"] = 1
treino["f_nacional"] = treino["f_nacional"].astype("category")
treino["f_regional"] = treino["f_regional"].astype("category")
treino["f_local"] = treino["f_local"].astype("category")


# oil
for i in range(len(gasolina["date"])):
    mask = gasolina["date"].iloc[i] == treino["date"]
    treino["gasolina"][mask] = gasolina["dcoilwtico"].loc[i]

mask = treino["gasolina"] == 0
treino.loc[mask, "gasolina"] = np.nan
treino["gasolina"].fillna(method='bfill', inplace=True)

# transactions
for date in transacoes["date"].unique():
    mask = transacoes[transacoes["date"] == date]
    for i in mask["store_nbr"]:
        if mask.loc[(mask["store_nbr"] == i), "transactions"].values[0] == 0: pass
        else:
            mask_venda = treino[(treino["date"] == date) & (treino["store_nbr"] == i)]
            vendas = mask_venda["sales"].sum()
            trans_loja = mask[mask["store_nbr"] == i]["transactions"]
            ticket_medio = vendas/trans_loja.values[0]
            filtro = treino[(treino["date"] == date) & (treino["store_nbr"] == i) & (treino["sales"] > 0)].index
            treino.loc[filtro, "ticket_medio"] = ticket_medio

# fim_de_semana (dummy)
for i in treino["date"].index:
    if (treino["date"][i].strftime('%A') == "Saturday") | (treino["date"][i].strftime('%A') == "Sunday"): treino.loc[i, "fim_de_semana"] = 1 
    else: pass

# pgto (indicador ordinal regressivo discreto do dia de pagamento no dia 15)
for i in treino.index:
        # dia 15
    if treino.loc[i, "date"].strftime("%d") != "15": pass
    else: treino.loc[i, "pgto"] = 6
        # dia 16
    if treino.loc[i, "date"].strftime("%d") != "16": pass
    else: treino.loc[i, "pgto"] = 5
        # dia 17
    if treino.loc[i, "date"].strftime("%d") != "17": pass
    else: treino.loc[i, "pgto"] = 4
        # dia 18
    if treino.loc[i, "date"].strftime("%d") != "18": pass
    else: treino.loc[i, "pgto"] = 3
        # dia 19
    if treino.loc[i, "date"].strftime("%d") != "19": pass
    else: treino.loc[i, "pgto"] = 2
        # dia 20
    if treino.loc[i, "date"].strftime("%d") != "20": pass
    else: treino.loc[i, "pgto"] = 1

# pgto (indicador ordinal regressivo discreto do dia de pagamento no último dia do mês)
for i in treino.index:
    ultimo_dia = calendar.monthrange(treino.loc[i, "date"].year, treino.loc[i, "date"].month)[1]
        # ultimo dia do mês em curso
    if treino.loc[i, "date"].strftime("%d") != str(ultimo_dia): pass
    else: treino.loc[i, "pgto"] = 6
        # primeiro dia do mês em curso
    if treino.loc[i, "date"].strftime("%d") != "01": pass
    else: treino.loc[i, "pgto"] = 5
        # dia 02
    if treino.loc[i, "date"].strftime("%d") != "02": pass
    else: treino.loc[i, "pgto"] = 4
        # dia 03
    if treino.loc[i, "date"].strftime("%d") != "03": pass
    else: treino.loc[i, "pgto"] = 3
        # dia 04
    if treino.loc[i, "date"].strftime("%d") != "04": pass
    else: treino.loc[i, "pgto"] = 2
        # dia 05
    if treino.loc[i, "date"].strftime("%d") != "05": pass
    else: treino.loc[i, "pgto"] = 1

# Dados de Seasonality
dia_da_semana = treino["date"].dt.dayofweek.copy()
mes_do_ano = treino["date"].dt.month.copy()
#trimestre = treino["date"].dt.quarter.copy()
treino["dia_da_semana"] = dia_da_semana
treino["mes_do_ano"] = mes_do_ano
treino["trimestre"] = trimestre

    
###
# Dataset de teste
### 

# store_type
for i in range(len(teste["store_type"])):
    teste["store_type"].iloc[i] = lojas["type"][teste["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# cluster
for i in range(len(teste["cluster"])):
    teste["cluster"].iloc[i] = lojas["cluster"][teste["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

    
# holidays_events
teste["f_nacional"] = 0
teste["f_regional"] = 0
teste["f_local"] = 0
for i in range(len(feriados)):
    if feriados.loc[i, "f_nacional"] == 1:
        mask = teste["date"] == feriados.loc[i, "date"]
        teste.loc[mask, "f_nacional"] = 1
    elif feriados.loc[i, "f_regional"] == 1:
        mask = teste["date"] == feriados.loc[i, "date"]
        teste.loc[mask, "f_regional"] = 1
    elif feriados.loc[i, "f_local"] == 1:
        mask = teste["date"] == feriados.loc[i, "date"]
        teste.loc[mask, "f_local"] = 1
teste["f_nacional"] = teste["f_nacional"].astype("category")
teste["f_regional"] = teste["f_regional"].astype("category")
teste["f_local"] = teste["f_local"].astype("category")


# oil
for i in range(len(gasolina["date"])):
    mask = gasolina["date"].iloc[i] == teste["date"]
    teste["gasolina"][mask] = gasolina["dcoilwtico"].loc[i]

mask = teste["gasolina"] == 0
teste.loc[mask, "gasolina"] = np.nan
teste["gasolina"].fillna(method='bfill', inplace=True)

# fim_de_semana (dummy)
for i in teste["date"].index:
    if (teste["date"][i].strftime('%A') == "Saturday") | (teste["date"][i].strftime('%A') == "Sunday"): teste.loc[i, "fim_de_semana"] = 1 
    else: pass

# pgto (indicador ordinal regressivo discreto do dia de pagamento)
for i in teste.index:
        # dia 15
    if teste.loc[i, "date"].strftime("%d") != "15": pass
    else: teste.loc[i, "pgto"] = 6
        # dia 16
    if teste.loc[i, "date"].strftime("%d") != "16": pass
    else: teste.loc[i, "pgto"] = 5
        # dia 17
    if teste.loc[i, "date"].strftime("%d") != "17": pass
    else: teste.loc[i, "pgto"] = 4
        # dia 18
    if teste.loc[i, "date"].strftime("%d") != "18": pass
    else: teste.loc[i, "pgto"] = 3
        # dia 19
    if teste.loc[i, "date"].strftime("%d") != "19": pass
    else: teste.loc[i, "pgto"] = 2
        # dia 20
    if teste.loc[i, "date"].strftime("%d") != "20": pass
    else: teste.loc[i, "pgto"] = 1


# Dados de Seasonality
dia_da_semana = teste["date"].dt.dayofweek.copy()
mes_do_ano = teste["date"].dt.month.copy()
trimestre = teste["date"].dt.quarter.copy()
teste["dia_da_semana"] = dia_da_semana
teste["mes_do_ano"] = mes_do_ano
teste["trimestre"] = trimestre

###
# Limpeza do workspace
### 
del (mask, feriado, filtro, ticket_medio, trans_loja, vendas, mask_venda, copia, data_teste, date, i, j, dia_da_semana, mes_do_ano, trimestre)

"""

'\n###\n# Dataset de treino\n### \n\n# store_type\nfor i in range(len(treino["store_type"])):\n    treino["store_type"].iloc[i] = lojas["type"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]\n\n# cluster\nfor i in range(len(treino["cluster"])):\n    treino["cluster"].iloc[i] = lojas["cluster"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]\n\n    \n# holidays_events\ntreino["f_nacional"] = 0\ntreino["f_regional"] = 0\ntreino["f_local"] = 0\nfor i in range(len(feriados)):\n    if feriados.loc[i, "f_nacional"] == 1:\n        mask = treino["date"] == feriados.loc[i, "date"]\n        treino.loc[mask, "f_nacional"] = 1\n    elif feriados.loc[i, "f_regional"] == 1:\n        mask = treino["date"] == feriados.loc[i, "date"]\n        treino.loc[mask, "f_regional"] = 1\n    elif feriados.loc[i, "f_local"] == 1:\n        mask = treino["date"] == feriados.loc[i, "date"]\n        treino.loc[mask, "f_local"] = 1\ntreino["f_nacional"] = treino["f_nacional"].astype("categor

### Importação dos dados (parquet)

In [4]:
# Dados com outliers
treino = pd.read_parquet("./parquet/treino.parquet")
teste = pd.read_parquet("./parquet/teste.parquet")
#feriados = pd.read_parquet("./parquet/feriados.parquet") # mantidos para caso seja necessário importara cada um individualmente
#gasolina = pd.read_parquet("./parquet/gasolina.parquet")
#transacoes = pd.read_parquet("./parquet/transacoes.parquet")
#lojas = pd.read_parquet("./parquet/lojas.parquet")

## Gráficos, Outliers e Normalização

### Bibliotecas e funções necessárias

In [5]:
from sklearn.model_selection import TimeSeriesSplit # separação de grupo treino e teste
from sklearn.metrics import root_mean_squared_log_error
import xgboost as xgb # https://xgboost.readthedocs.io/en/stable/parameter.html
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor

from typing import Tuple

#### Definição das funções necessárias

In [6]:
def gradient(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Computa o gradient squared log error.'''
    y = xgb.DMatrix(dtrain).get_label()
    return (np.log1p(predt) - np.log1p(y)) / (predt + 1)

def hessian(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Computa o hessiano para o squared log error.'''
    y = dtrain.get_label()
    return ((-np.log1p(predt) + np.log1p(y) + 1) /
            np.power(predt + 1, 2))

def squared_log(predt: np.ndarray,
                dtrain: xgb.DMatrix) -> Tuple[np.ndarray, np.ndarray]:
    '''Squared Log Error. Versão mais simples da RMSLE usada como função objetiva'''
    predt[predt < -1] = -1 + 1e-6
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

def rmsle(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    ''' Métrica RMSLE '''
    y = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    elements = np.power(np.log1p(y) - np.log1p(predt), 2)
    return 'PyRMSLE', float(np.sqrt(np.sum(elements) / len(y)))

#### Matriz de Correlação (geral)

Percebe-se uma alta correlação positiva entre as variáveis 'trimestre' e 'mes_do_ano', optando-se por desconsiderar-se, então, a variável 'trimestre'.

In [7]:
# Essa matriz utiliza o objeto gerado na seção "União Treino/Teste para criação das variáveis temporais".
"""
#df = treino.drop(["date", "family", "store_nbr", "cluster", "store_type", "id"], axis = 1).copy()
df = temp_18m.drop(["family", "store_nbr", "cluster", "store_type", "id"], axis = 1).copy()
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')
"""

'\n#df = treino.drop(["date", "family", "store_nbr", "cluster", "store_type", "id"], axis = 1).copy()\ndf = temp_18m.drop(["family", "store_nbr", "cluster", "store_type", "id"], axis = 1).copy()\ncorr = df.corr()\ncorr.style.background_gradient(cmap=\'coolwarm\')\n'

#### Outliers

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
color_pal = sns.color_palette()

##### Visualização das variáveis "gasolina" (treino/teste) e "sales"

        "sales" e "gasolina"

A primeira variável ajustada somente no banco de dados de treino.

In [9]:
"""
treino.plot(x = "date",
            y = "gasolina",
            ylabel = "Custo monetário",
            xlabel = "",
            label = "Custo da gasolina",
            figsize = (15,5),
            color = color_pal[0],
            title = "Gráfico 01 - Valor da gasolina ao longo do tempo\nDataset Treino")

teste.plot(x = "date",
            y = "gasolina",
            ylabel = "Custo monetário",
            xlabel = "",
            label = "Custo da gasolina",
            figsize = (15,5),
            color = color_pal[0],
            title = "Gráfico 02 - Valor da gasolina ao longo do tempo\nDataset Teste")

treino.plot(x = "date",
            y = "sales",
            ylabel = "Valor de vendas",
            xlabel = "",
            label = "Valor em vendas na data",
            style = '.',
            figsize = (15,5),
            color = color_pal[1],
            title = "Gráfico 03 - Vendas ao longo do tempo")

plt.show()
"""

'\ntreino.plot(x = "date",\n            y = "gasolina",\n            ylabel = "Custo monetário",\n            xlabel = "",\n            label = "Custo da gasolina",\n            figsize = (15,5),\n            color = color_pal[0],\n            title = "Gráfico 01 - Valor da gasolina ao longo do tempo\nDataset Treino")\n\nteste.plot(x = "date",\n            y = "gasolina",\n            ylabel = "Custo monetário",\n            xlabel = "",\n            label = "Custo da gasolina",\n            figsize = (15,5),\n            color = color_pal[0],\n            title = "Gráfico 02 - Valor da gasolina ao longo do tempo\nDataset Teste")\n\ntreino.plot(x = "date",\n            y = "sales",\n            ylabel = "Valor de vendas",\n            xlabel = "",\n            label = "Valor em vendas na data",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[1],\n            title = "Gráfico 03 - Vendas ao longo do tempo")\n\nplt.show()\n'

##### Visualização das vendas por loja

In [10]:
"""
treino[treino["date"] >= "2017-04-01"].plot(x = "date",
            y = "sales",
            ylabel = "Valor de vendas",
            xlabel = "",
            label = "Valor em vendas na data",
            style = '.',
            figsize = (15,5),
            color = color_pal[1],
            title = "Gráfico 03 - Vendas ao longo do tempo")

plt.show()
"""

'\ntreino[treino["date"] >= "2017-04-01"].plot(x = "date",\n            y = "sales",\n            ylabel = "Valor de vendas",\n            xlabel = "",\n            label = "Valor em vendas na data",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[1],\n            title = "Gráfico 03 - Vendas ao longo do tempo")\n\nplt.show()\n'

##### Visualização detalhada da variável "onpromotion" (treino/teste)

Na variável "onpromotion" eu realizo a análise dos outliers em momentos distintos e claramente identificáveis através dos gráficos.

Períodos de separação para análise dos outliers:
- desde o início dos registros até 31/03/2014 (onde 'onpromotion' é igual a zero sempre);
- de 01/04/2014 até o final desse ano (nada gritante em termos de outliers, apenas promoções sazonais);
- 2015 possui o primeiro semestre com menor quantidade de promoções em relação ao segundo semestre, mas mesmo assim sem outliers;
- 2016 é o ano do terremoto, onde sim fica gritante a presença de outliers;
- 2017 apresenta basicamente promoções regulares e sazonais, mas alguns outliers são identificados e também serão removidos.
- dataset de teste também apresenta visualmente alguns outliers no final do período, marcados para remoção.

In [11]:
"""
teste.plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[9],
            title = "Gráfico 04 - Promoções durante todo o período das observações\nDataset Teste")

treino.plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[2],
            title = "Gráfico 05 - Promoções durante todo o período das observações\nDataset Treino")

treino[treino["date"] < "2014-04-01"].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[4],
            title = "Gráfico 06 - Promoções do início das observações\naté 31/03/2014 (inclusive)")

treino[(treino["date"] >= "2014-04-01") & (treino["date"] <= "2014-12-31")].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[4],
            title = "Gráfico 07 - Promoções de 2014 a partir de 01/04")

treino[(treino["date"] >= "2015-01-01") & (treino["date"] <= "2015-12-31")].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[5],
            title = "Gráfico 08 - Promoções ao longo de todo 2015")

treino[(treino["date"] >= "2016-01-01") & (treino["date"] <= "2016-12-31")].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[7],
            title = "Gráfico 09 - Promoções ao longo de todo 2016")

treino[(treino["date"] >= "2016-04-16") & (treino["date"] <= "2016-06-15")].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[7],
            title = "Gráfico 10 - Visualização amplidada do período\nde 16/04/2016 a 15/06/2016 (época do terremoto)")

treino[treino["date"] >= "2017-01-01"].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[8],
            title = "Gráfico 11 - Promoções ao longo de todo 2017 observado")

plt.show()
"""

'\nteste.plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[9],\n            title = "Gráfico 04 - Promoções durante todo o período das observações\nDataset Teste")\n\ntreino.plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[2],\n            title = "Gráfico 05 - Promoções durante todo o período das observações\nDataset Treino")\n\ntreino[treino["date"] < "2014-04-01"].plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[4],

In [12]:
"""
terremoto_df = treino[(treino["date"] >= "2016-04-16") & (treino["date"] <= "2016-06-08")].copy()
terremoto_df.plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[8],
            title = "Réplica do Gráfico 10:\nPromoções ao longo do período do terremoto")

plt.show()
"""

'\nterremoto_df = treino[(treino["date"] >= "2016-04-16") & (treino["date"] <= "2016-06-08")].copy()\nterremoto_df.plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[8],\n            title = "Réplica do Gráfico 10:\nPromoções ao longo do período do terremoto")\n\nplt.show()\n'

##### Tratamento dos outliers

In [13]:

###
# Outliers variável "onpromotion" no ano do terremoto (2016)
###
terremoto_df = treino[(treino["date"] >= "2016-01-01") & (treino["date"] <= "2016-12-31")].copy()
max_terremoto = terremoto_df[terremoto_df["date"] == "2016-06-08"]["onpromotion"].max() # definida a data de 08/06 com base na dispersão das observações no Gráfico 10
apagar = terremoto_df[terremoto_df["onpromotion"] > max_terremoto].index
treino = treino.drop(apagar, axis = 0)


###
# Outliers variável "onpromotion" no ano de 2017
###
ajuste_df = treino[(treino["date"] >= "2017-01-01") & (treino["date"] <= "2017-08-15")].copy()
max_ajuste = ajuste_df[ajuste_df["date"] == "2017-03-15"]["onpromotion"].max() # definida a data de 15/03, com o máximo de 247
apagar = ajuste_df[ajuste_df["onpromotion"] > max_ajuste].index
treino = treino.drop(apagar, axis = 0)


###
# Outliers variável "sales"
###
max_ajuste = treino[treino["date"] == "2013-02-14"]["sales"].max() # definida a data de 14/02, com o máximo de 26067
apagar = treino[treino["sales"] > max_ajuste].index
treino = treino.drop(apagar, axis = 0)
"""
 
treino[(treino["date"] >= "2016-04-16") & (treino["date"] <= "2016-06-15")].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[7],
            title = "Gráfico 10 (revisitado) - Visualização amplidada\ndo período de 16/04/2016 a 15/06/2016 (época do terremoto)")

treino[treino["date"] >= "2017-01-01"].plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[7],
            title = "Gráfico 11 (revisitado) - Promoções ao longo de todo 2017 observado")

teste.plot(x = "date",
            y = "onpromotion",
            ylabel = "Qtde de promoções",
            xlabel = "",
            label = "Promoções",
            style = '.',
            figsize = (15,5),
            color = color_pal[9],
            title = "Gráfico 04 (revisitado) - Promoções durante todo o período das observações\nDataset Teste")

treino.plot(x = "date",
            y = "sales",
            ylabel = "Valor de vendas",
            xlabel = "",
            label = "Valor em vendas na data",
            style = '.',
            figsize = (15,5),
            color = color_pal[1],
            title = "Gráfico 03 (revisitado) - Vendas ao longo do tempo, sem outliers")

plt.show()
"""

'\n \ntreino[(treino["date"] >= "2016-04-16") & (treino["date"] <= "2016-06-15")].plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[7],\n            title = "Gráfico 10 (revisitado) - Visualização amplidada\ndo período de 16/04/2016 a 15/06/2016 (época do terremoto)")\n\ntreino[treino["date"] >= "2017-01-01"].plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções",\n            style = \'.\',\n            figsize = (15,5),\n            color = color_pal[7],\n            title = "Gráfico 11 (revisitado) - Promoções ao longo de todo 2017 observado")\n\nteste.plot(x = "date",\n            y = "onpromotion",\n            ylabel = "Qtde de promoções",\n            xlabel = "",\n            label = "Promoções"

#### Normalização

Normaliza as variáveis _sales_, _onpromotion_ e _gasolina_, depois de concatenar treino e teste. Disponível nos datasets *temp_zscore* e *temp_log*.

Ajustar nos datasets para modelagem conforme necessário.

In [14]:
from sklearn.preprocessing import StandardScaler # normalização por z-score
from sklearn.preprocessing import FunctionTransformer # normalização por log

In [15]:
a_normalizar = ["sales", "onpromotion", "gasolina", "pgto"]
temp = pd.concat([treino, teste])
temp.reset_index(inplace = True, drop = True)
temp["sales"] = temp["sales"].fillna(0)
temp_norm = temp[a_normalizar].copy()

# Normalizaçao por Z-Score
zscore_scaler = StandardScaler()
temp_zscore = zscore_scaler.fit_transform(temp_norm)
temp_zscore = pd.DataFrame(temp_zscore)
temp_zscore.columns = temp_norm.columns

# Normalização por Log
log_scaler = FunctionTransformer(np.log1p)
temp_log = log_scaler.fit_transform(temp_norm)

temp_snorm = temp.copy()
temp.drop(a_normalizar, axis = 1, inplace = True)
temp_normz = pd.concat([temp, temp_zscore], axis = 1)
temp_norml = pd.concat([temp, temp_log], axis = 1)

## Modelagem

#### União treino/teste para criação das variáveis temporais

In [124]:
# Criação de objeto das variáveis sem normalização para referência ao comparar os resultados da modelagem na apresentação final
temp_snorm = pd.concat([treino, teste])

### Descomentar a linha abaixo para utilização de modelagem com valores sem normalização
temp = temp_snorm.copy()

### Descomentar a linha abaixo para utilização de modelagem com valores normalizados via Z-Score
#temp = temp_normz.copy()

### Descomentar a linha abaixo para utilização de modelagem com valores normalizados via Log
#temp = temp_norml.copy()

# Ajuste da coluna 'sales' no dataset agrupado, em função do banco de dados de teste não ter essa informação
temp["sales"] = temp["sales"].fillna(0)

# Criação das variáveis de lag (D-n)
temp["sales_lag15"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].shift(15)
temp["sales_lag10"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].shift(10)
temp["sales_lag5"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].shift(5)
temp["sales_lag1"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].shift(1)

# Criação das variáveis de diferenças
temp["diff15"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].diff(15)
temp["diff10"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].diff(10)
temp["diff5"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].diff(5)
temp["diff1"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].diff(1)

# Apenas registro histórico, se quiser utilizar variável de média móvel - não teve significância em praticamente nenhumm modelo testado
#temp["mmovel15"] = temp.groupby(["family", "store_nbr"], observed = True, as_index = True)["sales"].rolling(15).mean().reset_index(level = 0, drop = True).values


# Inclusão de variável conjunta dos feriados nacional, regional e local
feriados = pd.read_csv("./csv/holidays_events.csv")
feriados["date"] = pd.to_datetime(feriados["date"])
for i in range(len(feriados)):
    if feriados.loc[i, "locale"] == "National" or feriados.loc[i, "locale"] == "Regional" or feriados.loc[i, "locale"] == "Local":
        feriados.loc[i, "feriado"] = 1
    else: pass

temp["feriado"] = 0

for i in range(len(feriados)):
    if feriados.loc[i, "feriado"] == 1:
        mask = temp.index == feriados.loc[i, "date"]
        temp.loc[mask, "feriado"] = 1

      
### Ajuste dos types das variáveis
temp["store_nbr"] = temp["store_nbr"].astype("category")
temp["cluster"] = temp["cluster"].astype("category")
temp["pgto"] = temp["pgto"].astype("category")
temp["dia_da_semana"] = temp["dia_da_semana"].astype("category")
temp["mes_do_ano"] = temp["mes_do_ano"].astype("category")
temp["f_nacional"] = temp["f_nacional"].astype("category")
temp["f_regional"] = temp["f_regional"].astype("category")
temp["f_local"] = temp["f_local"].astype("category")
temp["feriado"] = temp["feriado"].astype("category")


# Transformação do índice regular para as datas
temp.index = pd.DatetimeIndex(temp["date"])
temp.drop(["date", "trimestre"], axis = 1, inplace = True)


# Criação de fatias temporais mais recentes para o estudo, conforme preferência
#temp_6m = temp[temp.index > "2017-02-16"].copy()
#temp_12m = temp[temp.index > "2016-08-15"].copy()
#temp_18m = temp[temp.index > "2016-02-16"].copy()
temp_24m = temp[temp.index > "2015-08-15"].copy()


## Definição do banco de dados a ser usado 

# Banco de dados completo
#dados_treino = temp[temp["id"] < 3000888].copy()

# Banco de dados com os últimos 24 meses apenas
dados_treino = temp_24m[temp_24m["id"] < 3000888].copy()

# Banco de dados com os últimos 18 meses apenas
#dados_treino = temp_18m[temp_18m["id"] < 3000888].copy()

# Banco de dados com os últimos 12 meses apenas
#dados_treino = temp_12m[temp_12m["id"] < 3000888].copy()

# Banco de dados com os últimos 06 meses apenas
#dados_treino = temp_6m[temp_6m["id"] < 3000888].copy()

# Variáveis a remover da modelagem:
var_a_remover = ["id", "sales", "ticket_medio"]


In [94]:
# Registro à parte das variáveis para remoção por questão de facilidade enquanto se está modelando
#var_a_remover = ["id", "sales", "ticket_medio"]

#### Usando XGBoost e TimeSeriesSplit

##### Modelagem

In [135]:
var_a_remover = ["id", "sales", "ticket_medio"]

# Estabelecimento do nome do modelo para fins de registro
nome_modelo = datetime.now().strftime("%Y%m%d-%H%M")

# Avisa o tipo de normalização (se existente)
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR Z-SCORE\n")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nREALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO\n")
else:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR LOG\n")

# Registra o período dos dados em análise
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"), "\n")

# Informa sobre a existência de outliers no período de análise, tomando por base 'sales' da loja 9 em 02/04/2017 (observação recente arbitrada)
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: variáveis com presença de outliers!\n")
else:
    print("Parabéns, variáveis sem presença de outliers.\n")

# Variáveis independentes e dependente
y = ["sales"]
x = dados_treino.drop(var_a_remover, axis = 1).columns.values

# Criação do modelo de treino
tsgen = TimeSeriesSplit(n_splits=5, test_size=12474)

preds = []
score = []
scores_controle = []

for treino_ind, val_ind in tsgen.split(dados_treino):
    treino_reg = dados_treino.iloc[treino_ind]
    validacao_reg = dados_treino.iloc[val_ind]

    x_treino = treino_reg[x]
    y_treino = treino_reg[y]

    x_valid = validacao_reg[x]
    y_valid = validacao_reg[y]

    reg_xgb = xgb.XGBRegressor(base_score = 0.5, booster = "gbtree",
                           n_estimators = 1500,
                           early_stopping_rounds = 100,
                           objective = "reg:squarederror",
                           max_depth = 4,
                           learning_rate = 0.01,
                           enable_categorical = True, 
                           device = "gpu",
                           validate_parameters = True)

    
    reg_xgb.fit(x_treino, y_treino,
            eval_set = [(x_treino, y_treino)],
            verbose = 200)
    
    y_pred = reg_xgb.predict(x_valid)
    preds.append(y_pred)
    if (y_pred < 0).sum() > 0:
        score = "negativo"
    else:
        score = root_mean_squared_log_error(y_valid, y_pred)
    scores_controle.append(score)

discard_index = reg_xgb.feature_importances_ == 0

# Verifica a existência de normalização nos dados
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nDADOS COM NORMALIZAÇÃO POR Z-SCORE")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nDADOS SEM NORMALIZAÇÃO")
else:
    print("\nDADOS COM NORMALIZAÇÃO POR LOG")

# Verifica a existência de variáveis desprezíveis utilizadas na modelagem
if discard_index.sum() > 0:
    print("\nATENÇÃO! Variáveis sem importância encontradas!")
    print("\nPodem ser descartadas as variáveis: ", dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nTodas variáveis utilizadas, nenhuma sem importância.")


REALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO

Período de análise entre 16-08-2015 e 15-08-2017 

Parabéns, variáveis sem presença de outliers.

[0]	validation_0-rmse:1354.32921
[200]	validation_0-rmse:261.65611
[400]	validation_0-rmse:131.47698
[600]	validation_0-rmse:111.77811
[800]	validation_0-rmse:104.75443
[1000]	validation_0-rmse:99.85010
[1200]	validation_0-rmse:96.59438
[1400]	validation_0-rmse:93.83231
[1499]	validation_0-rmse:92.39529


/home/thiago/.pyenv/versions/3.12.3/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [11:45:32] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:1354.21685
[200]	validation_0-rmse:261.49608
[400]	validation_0-rmse:131.50476
[600]	validation_0-rmse:111.77042
[800]	validation_0-rmse:104.44623
[1000]	validation_0-rmse:99.67299
[1200]	validation_0-rmse:96.48048
[1400]	validation_0-rmse:94.07502
[1499]	validation_0-rmse:92.84417
[0]	validation_0-rmse:1353.86054
[200]	validation_0-rmse:261.36549
[400]	validation_0-rmse:131.22312
[600]	validation_0-rmse:111.36751
[800]	validation_0-rmse:104.17175
[1000]	validation_0-rmse:99.15316
[1200]	validation_0-rmse:95.94128
[1400]	validation_0-rmse:93.30612
[1499]	validation_0-rmse:92.14074
[0]	validation_0-rmse:1354.75219
[200]	validation_0-rmse:261.11754
[400]	validation_0-rmse:131.03174
[600]	validation_0-rmse:111.29385
[800]	validation_0-rmse:104.02178
[1000]	validation_0-rmse:99.28769
[1200]	validation_0-rmse:96.02516
[1400]	validation_0-rmse:93.03070
[1499]	validation_0-rmse:91.86168
[0]	validation_0-rmse:1355.01173
[200]	validation_0-rmse:261.27296
[400]	validation_0

##### Apresentação do modelo utilizado e registro dele em arquivo

In [136]:
ext = (".json")
save = ("modelo_xgb_"+(nome_modelo + ext))
reg_xgb.save_model("./modelos/"+save)
print("###### MODELO XGBoost " + nome_modelo + " ######")
print("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
print("Modelo utilizado salvo como", save)
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados com normalização por ***Z-SCORE***")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("Dados ***sem normalização***")
else:
    print("Dados com normalização por ***LOG***")
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"))
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: análise realizada com outliers presentes nas variáveis!")
else:
    print("Análise realizada sem presença de outliers nas variáveis.")
print("\nScores obtidos no controle:", scores_controle)
print("Média dos scores obtidos no controle:", pd.DataFrame(scores_controle).mean(numeric_only= True).values)
print("Mediana dos scores obtidos no controle:", pd.DataFrame(scores_controle).median(numeric_only= True).values)
print("Melhor score obtido no controle:", pd.DataFrame(scores_controle).min(numeric_only= True).values)
print("\nFeatures utilizadas no modelo e suas respectivas importâncias:")
importancias = pd.DataFrame(reg_xgb.feature_importances_, reg_xgb.feature_names_in_)
importancias.reset_index(inplace = True)
importancias.columns = ["variável", "importância"]
print(importancias.sort_values(by = "importância", ascending = False))
if discard_index.sum() > 0:
    print("\nFeatures sem importância:", dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nNão foram identificadas features sem importância.")
print("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
print("Tamanho de teste para cada split: " + str(tsgen.test_size) + " observações")
print("\nParâmetros do modelo:")
print("           base_score: " + str(reg_xgb.base_score))
print("              booster: " + str(reg_xgb.booster))
print("         n_estimators: " + str(reg_xgb.n_estimators))
print("early_stopping_rounds: " + str(reg_xgb.early_stopping_rounds))
print("            objective: " + reg_xgb.objective)
print("            max_depth: " + str(reg_xgb.max_depth))
print("        learning_rate: " + str(reg_xgb.learning_rate))
print("   enable_categorical: " + str(reg_xgb.enable_categorical))
print("               device: " + reg_xgb.device)
print("  validade_parameters: True")
print("\n=======================================================================")

###### MODELO XGBoost 20240626-1145 ######

Registro gerado em 26-06-2024 às 11:48.
Modelo utilizado salvo como modelo_xgb_20240626-1145.json
Dados ***sem normalização***
Período de análise entre 16-08-2015 e 15-08-2017
Análise realizada sem presença de outliers nas variáveis.

Scores obtidos no controle: [0.49420063711364787, 0.4907497728800386, 0.4949306789625742, 0.48522499521043705, 0.4991934836996374]
Média dos scores obtidos no controle: [0.49286]
Mediana dos scores obtidos no controle: [0.494201]
Melhor score obtido no controle: [0.485225]

Features utilizadas no modelo e suas respectivas importâncias:
         variável  importância
15     sales_lag1     0.529783
14     sales_lag5     0.187984
13    sales_lag10     0.076940
12    sales_lag15     0.073948
19          diff1     0.043914
18          diff5     0.032132
17         diff10     0.023881
16         diff15     0.014852
10  dia_da_semana     0.002680
1          family     0.002579
0       store_nbr     0.002427
6          

##### Gera as previsões e faz a análise delas

In [137]:
# Gera as previsões
print("Manipulando dados para as previsões...")
dados_teste = temp[temp["id"] >= 3000888].copy()
dados_teste = dados_teste.drop(var_a_remover, axis = 1)

print("Gerando as previsões...")
y_final_pre = reg_xgb.predict(dados_teste)

# Retorno dos valores normalizados
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados originais normalizados por Z-Score, retornando valores à normalidade.")
#    y_final = zscore_scaler.inverse_transform(y_final_pre.reshape(-1, 1))
    y_final_pre = pd.DataFrame(y_final_pre)
    y_final_pre["a"] = 0
    y_final_pre["b"] = 0
    y_final_pre["c"] = 0
    y_final = pd.DataFrame(zscore_scaler.inverse_transform(y_final_pre))
    y_final.drop([1, 2], axis = 1, inplace = True)
    y_final = y_final.to_numpy(y_final[0])
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    y_final = y_final_pre.copy()
else:
    print("Dados originais normalizados por Log, restituindo à normalidade.")
    y_final = np.expm1(y_final_pre)

y_final_xgb = y_final.copy()

print("\nVerificando previsões de valores de venda negativos...")
if len(y_final_xgb[y_final_xgb < 0]) > 0:
    print("Convertendo valores negativos de venda em zero...")
    # Ajuste das previsões negativas, transformando o valor negativo para zero
    negativas = len(y_final_xgb[y_final_xgb < 0])
    y_final_xgb[y_final_xgb < 0] = 0
else:
    negativas = 0
    print("Sem previsões de vendas com valores negativos.")
print("\nPrevisões geradas com sucesso. Procedendo com a análise.\n")

vendas = round((y_final_xgb.sum() / temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum())*100, 2)
perda = 100 - vendas

print("*****")
print("Percentual de preenchimento de vendas:", vendas,"% dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)")
print("Total estimado de vendas futuras foi de", round(y_final_xgb.sum(), 2))
print("Valor de venda acumulado dos últimos 15 dias do dataset de treino:", round(temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum(), 2))
if perda < 0:
    print("Isso é o equivalente a um ganho teórico de",abs(perda), "% de faturamento.\n")
else: print("Isso é o equivalente a uma perda teórica de",round(perda, 2), "% de faturamento.")
print("*****")

# Análise das previsões feitas
print("Valor mínimo das vendas previstas: " + str(pd.DataFrame(y_final_xgb).min().values))
print("Valor da mediana das vendas previstas: " + str(pd.DataFrame(y_final_xgb).median().values))
print("Valor máximo das vendas previstas: " + str(pd.DataFrame(y_final_xgb).max().values))
if negativas == 0:
    print("Sem registro de previsões negativas através deste modelo")
else:
    print("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final))*100, 2)) + "% do total)")
print("\nRelação das frequências dos valores previstos:")
print("     -=+ valores mais frequentes +=-")
print(pd.DataFrame(y_final_xgb).value_counts().head(10))
print("     -=+ valores com menor frequência +=-")
print(pd.DataFrame(y_final_xgb).value_counts().tail(10))

Manipulando dados para as previsões...
Gerando as previsões...

Verificando previsões de valores de venda negativos...
Convertendo valores negativos de venda em zero...

Previsões geradas com sucesso. Procedendo com a análise.

*****
Percentual de preenchimento de vendas: 12.41 % dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)
Total estimado de vendas futuras foi de 1543109.9
Valor de venda acumulado dos últimos 15 dias do dataset de treino: 12433322.94
Isso é o equivalente a uma perda teórica de 87.59 % de faturamento.
*****
Valor mínimo das vendas previstas: [0.]
Valor da mediana das vendas previstas: [5.657938]
Valor máximo das vendas previstas: [6652.8774]
Quantidade de previsões negativas (tranformadas para zero): 252 (0.88% do total)

Relação das frequências dos valores previstos:
     -=+ valores mais frequentes +=-
0       
1.231609    731
2.360042    529
2.414225    525
1.892556    397
2.277859    257
0.000000    252
1.412441    193
2.758032    155
2.433170 

##### Registro em texto e gera arquivo de submissão

In [138]:
with open("registros.txt", "a") as registros:
    registros.write("################ INÍCIO DE REGISTRO - MODELO XGBoost " + nome_modelo + " ################\n")
    registros.write("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
    registros.write("\nModelo utilizado salvo como " + save)
    if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
        registros.write("\nDados com normalização por ***Z-SCORE***")
    elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
        registros.write("\nDados ***sem normalização***")
    else:
        registros.write("\nDados com normalização por ***LOG***")
    registros.write("\nPeríodo de análise entre " + str(dados_treino.index.min().strftime("%d-%m-%Y")) + " e " + str(dados_treino.index.max().strftime("%d-%m-%Y")) + ". \n")
    if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
        registros.write("Atenção: análise realizada com outliers presentes nas variáveis!\n")
    else:
        registros.write("Análise realizada sem presença de outliers nas variáveis.\n")
    registros.write("\nScores obtidos no controle: " + str(scores_controle))
    if "negativo" in scores_controle:
        registros.write("\nControle possui score(s) negativo(s), sem média.")
        registros.write("\nControle possui score(s) negativo(s), sem mediana.")
        registros.write("\nControle possui score(s) negativo(s), sem registro de melhor score.\n")
    else:
        registros.write("\nMédia dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).mean().values))
        registros.write("\nMediana dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).median().values))
        registros.write("\nMelhor score obtido no controle:" + str(pd.DataFrame(scores_controle).min().values) + "\n")
    registros.write("\nFeatures utilizadas no modelo e seus respectivos impactos:\n")
    importancias = pd.DataFrame(reg_xgb.feature_importances_, reg_xgb.feature_names_in_)
    importancias.reset_index(inplace = True)
    importancias.columns = ["variável", "importância"]
    importancias = importancias.sort_values(by = "importância", ascending = False)
    registros.write(importancias.to_string(index = False) + "\n")
    if discard_index.sum() > 0:
        registros.write("\nFeatures sem importância: "+ str(dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index]) + "\n")
    else: registros.write("\nNão foram identificadas features sem importância. \n")
    registros.write("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
    registros.write("\nTamanho de teste para cada split: " + str(tsgen.test_size) + " observações (dias?)\n")
    registros.write("\nParâmetros do modelo:\n")
    registros.write("           base_score: " + str(reg_xgb.base_score) + "\n")
    registros.write("              booster: " + str(reg_xgb.booster) + "\n")
    registros.write("         n_estimators: " + str(reg_xgb.n_estimators) + "\n")
    registros.write("early_stopping_rounds: " + str(reg_xgb.early_stopping_rounds) + "\n")
    registros.write("            objective: " + reg_xgb.objective+ "\n")
    registros.write("            max_depth: " + str(reg_xgb.max_depth) + "\n")
    registros.write("        learning_rate: " + str(reg_xgb.learning_rate) + "\n")
    registros.write("   enable_categorical: " + str(reg_xgb.enable_categorical) + "\n")
    registros.write("               device: " + reg_xgb.device + "\n")
    registros.write("  validade_parameters: True" + "\n")
    registros.write("\n")
    registros.write(" ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO MODELO +++===---\n")
    registros.write("\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_xgb).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_xgb).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_xgb).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final))*100, 2)) + "% do total).\n")

    registros.write("\nRelação das frequências dos valores previstos:\n")
    registros.write("     -=+ valores mais frequentes +=-\n")
    maisfreq = pd.DataFrame(y_final_xgb).value_counts().head(10)
    registros.write(maisfreq.to_string() + "\n")
    registros.write("     -=+ valores com menor frequência +=-\n")
    menosfreq = (pd.DataFrame(y_final_xgb).value_counts().tail(10))
    registros.write(menosfreq.to_string() + "\n")
    registros.write("\n################ FINAL DE REGISTRO - MODELO XGBoost " + nome_modelo + " ################\n")
    registros.write("\n")
    registros.write("\n")

ext_final = (".csv")
save_final = ("resultado_XGB_"+(nome_modelo + ext_final))
submissao = pd.read_csv("./csv/sample_submission.csv")
submissao["sales"] = y_final_xgb
resultado = submissao[["id", "sales"]]
resultado.to_csv("./resultados/"+save_final, index=False)

#### Usando DecisionTree e TimeSeriesSplit

##### Modelagem

In [131]:
var_a_remover = ["id", "sales", "ticket_medio"]

# Estabelecimento do nome do modelo para fins de registro
nome_modelo = datetime.now().strftime("%Y%m%d-%H%M")

# Avisa o tipo de normalização (se existente)
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR Z-SCORE\n")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nREALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO\n")
else:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR LOG\n")

# Registra o período dos dados em análise
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"), "\n")

# Informa sobre a existência de outliers no período de análise, tomando por base 'sales' da loja 9 em 02/04/2017 (observação recente arbitrada)
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: variáveis com presença de outliers!\n")
else:
    print("Parabéns, variáveis sem presença de outliers.\n")

# Variáveis independentes e dependente
categoricas_dt = ['store_type', 'family']
dados_treino_dt = dados_treino.drop(categoricas_dt, axis = 1)
y = ["sales"]
x = dados_treino_dt.drop(var_a_remover, axis = 1).columns.values

# Criação do modelo de treino
tsgen = TimeSeriesSplit(n_splits=6, test_size=12474)

preds = []
score = []
scores_controle = []

for treino_ind, val_ind in tsgen.split(dados_treino_dt):
    treino_reg = dados_treino_dt.iloc[treino_ind]
    validacao_reg = dados_treino_dt.iloc[val_ind]

    x_treino = treino_reg[x]
    y_treino = treino_reg[y]

    x_valid = validacao_reg[x]
    y_valid = validacao_reg[y]

    reg_dt = DecisionTreeRegressor(max_depth = 5,
                                   criterion = 'poisson',
                                   random_state=1)
    
    reg_dt.fit(x_treino, y_treino)
    
    y_pred = reg_dt.predict(x_valid)
    preds.append(y_pred)
    if (y_pred < 0).sum() > 0:
        score = "negativo"
    else:
        score = root_mean_squared_log_error(y_valid, y_pred)
    scores_controle.append(score)

discard_index = reg_dt.feature_importances_ == 0

# Verifica a existência de normalização nos dados
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nDADOS COM NORMALIZAÇÃO POR Z-SCORE")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nDADOS SEM NORMALIZAÇÃO")
else:
    print("\nDADOS COM NORMALIZAÇÃO POR LOG")

# Verifica a existência de variáveis desprezíveis utilizadas na modelagem
if discard_index.sum() > 0:
    print("\nATENÇÃO! Variáveis sem importância encontradas!")
    print("\nPodem ser descartadas as variáveis: ", dados_treino_dt.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nTodas variáveis utilizadas, nenhuma sem importância.")

# Realiza o cálculo do coeficiente de determinação R2 e apresenta ao usuário
coef_dt = reg_dt.score(x_valid, y_valid)
print("\nCoeficiente de determinação R2 do modelo: ", round(coef_dt, 2))



REALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO

Período de análise entre 16-08-2015 e 15-08-2017 

Parabéns, variáveis sem presença de outliers.


DADOS SEM NORMALIZAÇÃO

ATENÇÃO! Variáveis sem importância encontradas!

Podem ser descartadas as variáveis:  ['store_nbr' 'onpromotion' 'cluster' 'gasolina' 'pgto' 'f_nacional'
 'f_regional' 'f_local' 'dia_da_semana' 'mes_do_ano' 'sales_lag10'
 'sales_lag5' 'diff10' 'diff5' 'feriado']

Coeficiente de determinação R2 do modelo:  0.95


##### Apresentação do modelo utilizado

In [132]:
print("###### MODELO DecisionTree " + nome_modelo + " ######")
print("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
print("Modelo utilizado **não salvo**!")
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados com normalização por ***Z-SCORE***")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("Dados ***sem normalização***")
else:
    print("Dados com normalização por ***LOG***")
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"))
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: análise realizada com outliers presentes nas variáveis!")
else:
    print("Análise realizada sem presença de outliers nas variáveis.")

print("\nScores obtidos no controle:", scores_controle)
print("Média dos scores obtidos no controle:", pd.DataFrame(scores_controle).mean(numeric_only= True).values)
print("Mediana dos scores obtidos no controle:", pd.DataFrame(scores_controle).median(numeric_only= True).values)
print("Melhor score obtido no controle:", pd.DataFrame(scores_controle).min(numeric_only= True).values)
print("\nCoeficiente de determinação R2 do modelo: ", round(coef_dt, 2))

print("\nFeatures utilizadas no modelo e suas respectivas importâncias:")
importancias = pd.DataFrame(reg_dt.feature_importances_, reg_dt.feature_names_in_)
importancias.reset_index(inplace = True)
importancias.columns = ["variável", "importância"]
print(importancias.sort_values(by = "importância", ascending = False))
if discard_index.sum() > 0:
    print("\nFeatures sem importância:", dados_treino_dt.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nNão foram identificadas features sem importância.")
print("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
print("Tamanho de teste para cada split: " + str(tsgen.test_size) + " observações")
print("\nParâmetros do modelo:")
print(reg_dt.get_params())
print("\n=======================================================================")

###### MODELO DecisionTree 20240626-1142 ######

Registro gerado em 26-06-2024 às 11:43.
Modelo utilizado **não salvo**!
Dados ***sem normalização***
Período de análise entre 16-08-2015 e 15-08-2017
Análise realizada sem presença de outliers nas variáveis.

Scores obtidos no controle: [0.5312660569435057, 0.5362650756094832, 0.5248115966057562, 0.5231279016566045, 0.5065226131479291, 0.5089691230688885]
Média dos scores obtidos no controle: [0.521827]
Mediana dos scores obtidos no controle: [0.52397]
Melhor score obtido no controle: [0.506523]

Coeficiente de determinação R2 do modelo:  0.95

Features utilizadas no modelo e suas respectivas importâncias:
         variável  importância
13     sales_lag1     0.853001
10    sales_lag15     0.112846
17          diff1     0.028103
14         diff15     0.006050
0       store_nbr     0.000000
16          diff5     0.000000
15         diff10     0.000000
12     sales_lag5     0.000000
11    sales_lag10     0.000000
9      mes_do_ano     0.000

##### Gera as previsões e faz a análise delas

In [133]:
# Gera as previsões do modelo
print("Manipulando dados para as previsões...")
dados_teste = temp[temp["id"] >= 3000888].copy()
dados_teste.drop(categoricas_dt, axis = 1, inplace = True)
dados_teste.drop(var_a_remover, axis = 1, inplace = True)

print("Gerando as previsões...")
y_final_pre = reg_dt.predict(dados_teste)

# Retorno dos valores normalizados, se for o caso
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados originais normalizados por Z-Score, retornando valores à normalidade.")
#    y_final = zscore_scaler.inverse_transform(y_final_pre.reshape(-1, 1))
    y_final_pre = pd.DataFrame(y_final_pre)
    y_final_pre["a"] = 0
    y_final_pre["b"] = 0
    y_final_pre["c"] = 0
    y_final = pd.DataFrame(zscore_scaler.inverse_transform(y_final_pre))
    y_final.drop([1, 2], axis = 1, inplace = True)
    y_final = y_final.to_numpy(y_final[0])
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    y_final = y_final_pre.copy()
else:
    print("Dados originais normalizados por Log, restituindo à normalidade.")
    y_final = np.expm1(y_final_pre)

y_final_dt = y_final.copy()

print("\nVerificando previsões de valores de venda negativos...")
if len(y_final_dt[y_final_dt < 0]) > 0:
    print("Convertendo valores negativos de venda em zero...")
    # Ajuste das previsões negativas, transformando o valor negativo para zero
    negativas = len(y_final_dt[y_final_dt < 0])
    y_final_dt[y_final_dt < 0] = 0
else:
    negativas = 0
    print("Sem previsões de vendas com valores negativos.")
print("\nPrevisões geradas com sucesso. Procedendo com a análise.\n")

vendas = round((y_final_dt.sum() / temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum())*100, 2)
perda = 100 - vendas

print("*****")
print("Percentual de preenchimento de vendas:", vendas,"% dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)")
print("Total estimado de vendas futuras foi de", round(y_final_dt.sum(), 2))
print("Valor de venda acumulado dos últimos 15 dias do dataset de treino:", round(temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum(), 2))
if perda < 0:
    print("Isso é o equivalente a um ganho teórico de",abs(perda), "% de faturamento.\n")
else: print("Isso é o equivalente a uma perda teórica de",round(perda, 2), "% de faturamento.")
print("*****")

# Análise das previsões
print("Valor mínimo das vendas previstas: " + str(pd.DataFrame(y_final_dt).min().values))
print("Valor da mediana das vendas previstas: " + str(pd.DataFrame(y_final_dt).median().values))
print("Valor máximo das vendas previstas: " + str(pd.DataFrame(y_final_dt).max().values))
if negativas == 0:
    print("Sem registro de previsões negativas através deste modelo")
else:
    print("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_dt))*100, 2)) + "% do total)")
print("\nRelação das frequências dos valores previstos:")
print("     -=+ valores mais frequentes +=-")
print(pd.DataFrame(y_final_dt).value_counts().head(10))
print("     -=+ valores com menor frequência +=-")
print(pd.DataFrame(y_final_dt).value_counts().tail(10))

Manipulando dados para as previsões...
Gerando as previsões...

Verificando previsões de valores de venda negativos...
Sem previsões de vendas com valores negativos.

Previsões geradas com sucesso. Procedendo com a análise.

*****
Percentual de preenchimento de vendas: 26.59 % dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)
Total estimado de vendas futuras foi de 3306283.86
Valor de venda acumulado dos últimos 15 dias do dataset de treino: 12433322.94
Isso é o equivalente a uma perda teórica de 73.41 % de faturamento.
*****
Valor mínimo das vendas previstas: [0.75846]
Valor da mediana das vendas previstas: [0.75846]
Valor máximo das vendas previstas: [8425.015195]
Sem registro de previsões negativas através deste modelo

Relação das frequências dos valores previstos:
     -=+ valores mais frequentes +=-
0          
0.758460       17283
360.687924      5589
121.785006      2418
215.399350      2259
6.887592         401
28.889417        142
58.625001        107
569.469

##### Registro em texto e gera arquivo de submissão

In [134]:
with open("registros_dt.txt", "a") as registros:
    registros.write("################ INÍCIO DE REGISTRO - MODELO DecisionTree " + nome_modelo + " ################\n")
    registros.write("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
    registros.write("\nModelo utilizado **não salvo**!")
    if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
        registros.write("\nDados com normalização por ***Z-SCORE***")
    elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
        registros.write("\nDados ***sem normalização***")
    else:
        registros.write("\nDados com normalização por ***LOG***")
    registros.write("\nPeríodo de análise entre " + str(dados_treino_dt.index.min().strftime("%d-%m-%Y")) + " e " + str(dados_treino_dt.index.max().strftime("%d-%m-%Y")) + ". \n")
    if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
        registros.write("Atenção: análise realizada com outliers presentes nas variáveis!\n")
    else:
        registros.write("Análise realizada sem presença de outliers nas variáveis.\n")
    registros.write("\nScores obtidos no controle: " + str(scores_controle))
    if "negativo" in scores_controle:
        registros.write("\nControle possui score(s) negativo(s), sem média.")
        registros.write("\nControle possui score(s) negativo(s), sem mediana.")
        registros.write("\nControle possui score(s) negativo(s), sem registro de melhor score.\n")
    else:
        registros.write("\nMédia dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).mean().values))
        registros.write("\nMediana dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).median().values))
        registros.write("\nMelhor score obtido no controle:" + str(pd.DataFrame(scores_controle).min().values))
    registros.write("\nCoeficiente de determinação R2 do modelo:"+ str(round(coef_dt, 2)) + "\n")

    
    registros.write("\nFeatures utilizadas no modelo e seus respectivos impactos:\n")
    importancias = pd.DataFrame(reg_dt.feature_importances_, reg_dt.feature_names_in_)
    importancias.reset_index(inplace = True)
    importancias.columns = ["variável", "importância"]
    importancias = importancias.sort_values(by = "importância", ascending = False)
    registros.write(importancias.to_string(index = False) + "\n")
    if discard_index.sum() > 0:
        registros.write("\nFeatures sem importância: "+ str(dados_treino_dt.drop(var_a_remover, axis = 1).columns.values[discard_index]) + "\n")
    else: registros.write("\nNão foram identificadas features sem importância. \n")
    registros.write("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
    registros.write("\nTamanho de teste para cada split: " + str(tsgen.test_size) + " observações.\n")
    registros.write("\nParâmetros do modelo:\n")
    registros.write(str(reg_dt.get_params()))
    registros.write("\n")
    registros.write("\n ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO MODELO +++===---\n")
    registros.write("\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_dt).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_dt).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_dt).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_dt))*100, 2)) + "% do total).\n")

    registros.write("\nRelação das frequências dos valores previstos:\n")
    registros.write("     -=+ valores mais frequentes +=-\n")
    maisfreq = pd.DataFrame(y_final_dt).value_counts().head(10)
    registros.write(maisfreq.to_string() + "\n")
    registros.write("     -=+ valores com menor frequência +=-\n")
    menosfreq = (pd.DataFrame(y_final_dt).value_counts().tail(10))
    registros.write(menosfreq.to_string() + "\n")
    registros.write("\n################ FINAL DE REGISTRO - MODELO DecisionTree " + nome_modelo + " ################\n")
    registros.write("\n")
    registros.write("\n")

ext_final = (".csv")
save_final = ("resultado_dt_"+(nome_modelo + ext_final))
submissao = pd.read_csv("./csv/sample_submission.csv")
submissao["sales"] = y_final_dt
resultado = submissao[["id", "sales"]]
resultado.to_csv("./resultados/"+save_final, index=False)

#### Usando LightGBM e TimeSeriesSplit

##### Modelagem

In [88]:
var_a_remover = ["id", "sales", "ticket_medio",
                 'gasolina', 'f_nacional', 'cluster',
                 'f_regional', 'f_local', 'store_type', 'feriado']


# Estabelecimento do nome do modelo para fins de registro
nome_modelo = datetime.now().strftime("%Y%m%d-%H%M")

# Avisa o tipo de normalização (se existente)
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR Z-SCORE\n")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nREALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO\n")
else:
    print("\nREALIZANDO MODELAGEM, DADOS COM NORMALIZAÇÃO POR LOG\n")

# Registra o período dos dados em análise
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"), "\n")

# Informa sobre a existência de outliers no período de análise, tomando por base 'sales' da loja 9 em 02/04/2017 (observação recente arbitrada)
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: variáveis com presença de outliers!\n")
else:
    print("Parabéns, variáveis sem presença de outliers.\n")

# Variáveis independentes e dependente
y = ["sales"]
x = dados_treino.drop(var_a_remover, axis = 1).columns.values

# Criação do modelo de treino
tsgen = TimeSeriesSplit(n_splits=4, test_size=12474)

preds = []
score = []
scores_controle = []

for treino_ind, val_ind in tsgen.split(dados_treino):
    treino_reg = dados_treino.iloc[treino_ind]
    validacao_reg = dados_treino.iloc[val_ind]

    x_treino = treino_reg[x]
    y_treino = treino_reg[y]

    x_valid = validacao_reg[x]
    y_valid = validacao_reg[y]

    reg_lgbm = lgb.LGBMRegressor(boosting_type="gbdt", # gbdt (gradient boosting decision tree, padrão), dart (dropouts meet multiple additive regression trees), rf (random forest)
                                num_leaves = 45, # documentação fala em 2^max_depth
                                max_depth = 5,
                                learning_rate=0.01,
                                n_estimators=925,
                                objective='regression', # é o default
                                min_child_samples=30, # número mínimo de dados necessários para uma leaf (child)
                                random_state=1,
                                n_jobs=12,
                                metric="rmse")
                                #bagging_freq = 1,
                                #bagging_fraction = 0.9,
                                #bagging_seed = 1)

    
    reg_lgbm.fit(x_treino, y_treino)
    
    y_pred = reg_lgbm.predict(x_valid)
    preds.append(y_pred)
    if (y_pred < 0).sum() > 0:
        score = "negativo"
    else:
        score = root_mean_squared_log_error(y_valid, y_pred)
    scores_controle.append(score)

discard_index = reg_lgbm.feature_importances_ == 0

# Verifica a existência de normalização nos dados
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("\nDADOS COM NORMALIZAÇÃO POR Z-SCORE")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("\nDADOS SEM NORMALIZAÇÃO")
else:
    print("\nDADOS COM NORMALIZAÇÃO POR LOG")

# Verifica a existência de variáveis desprezíveis utilizadas na modelagem
if discard_index.sum() > 0:
    print("\nATENÇÃO! Variáveis sem importância encontradas!")
    print("\nPodem ser descartadas as variáveis: ", dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nTodas variáveis utilizadas, nenhuma sem importância.")

# Realiza o cálculo do coeficiente de determinação R2 e apresenta ao usuário
coef_lgbm = reg_lgbm.score(x_valid, y_valid)
print("\nCoeficiente de determinação R2 do modelo: ", round(coef_lgbm, 2))


REALIZANDO MODELAGEM, DADOS SEM NORMALIZAÇÃO

Período de análise entre 16-08-2015 e 15-08-2017 

Parabéns, variáveis sem presença de outliers.

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2384
[LightGBM] [Info] Number of data points in the train set: 1248938, number of used features: 14
[LightGBM] [Info] Start training from score 453.177756
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7e81a57e47c0>
Traceback (most recent call last):
  File "/home/thiago/.pyenv/versions/3.12.3/lib/python3.12/site-packages/lightgbm/basic.py", line 255, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

##### Apresentação do modelo utilizado

In [89]:
print("###### MODELO LGBM " + nome_modelo + " ######")
print("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
print("Modelo utilizado **não salvo**!")
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados com normalização por ***Z-SCORE***")
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    print("Dados ***sem normalização***")
else:
    print("Dados com normalização por ***LOG***")
print("Período de análise entre", dados_treino.index.min().strftime("%d-%m-%Y"), "e", dados_treino.index.max().strftime("%d-%m-%Y"))
if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
    print("Atenção: análise realizada com outliers presentes nas variáveis!")
else:
    print("Análise realizada sem presença de outliers nas variáveis.")

print("\nScores obtidos no controle:", scores_controle)
print("Média dos scores obtidos no controle:", pd.DataFrame(scores_controle).mean(numeric_only= True).values)
print("Mediana dos scores obtidos no controle:", pd.DataFrame(scores_controle).median(numeric_only= True).values)
print("Melhor score obtido no controle:", pd.DataFrame(scores_controle).min(numeric_only= True).values)
print("\nCoeficiente de determinação R2 do modelo: ", round(coef_lgbm, 2))

print("\nFeatures utilizadas no modelo e suas respectivas importâncias:")
importancias = pd.DataFrame(reg_lgbm.feature_importances_, reg_lgbm.feature_name_)
importancias.reset_index(inplace = True)
importancias.columns = ["variável", "importância"]
print(importancias.sort_values(by = "importância", ascending = False))
if discard_index.sum() > 0:
    print("\nFeatures sem importância:", dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index])
else: print("\nNão foram identificadas features sem importância.")
print("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
print("Tamanho de teste para cada split: " + str(tsgen.test_size) + " observações")
print("\nParâmetros do modelo:")
print(reg_lgbm.get_params())
print("\n=======================================================================")

###### MODELO LGBM 20240626-1055 ######

Registro gerado em 26-06-2024 às 10:57.
Modelo utilizado **não salvo**!
Dados ***sem normalização***
Período de análise entre 16-08-2015 e 15-08-2017
Análise realizada sem presença de outliers nas variáveis.

Scores obtidos no controle: [0.47772329269289393, 0.4757689937952239, 0.4796923392695586, 0.47277473277330523]
Média dos scores obtidos no controle: [0.47649]
Mediana dos scores obtidos no controle: [0.476746]
Melhor score obtido no controle: [0.472775]

Coeficiente de determinação R2 do modelo:  1.0

Features utilizadas no modelo e suas respectivas importâncias:
         variável  importância
13          diff1         6413
9      sales_lag1         4386
12          diff5         3289
11         diff10         2610
8      sales_lag5         2316
7     sales_lag10         1937
10         diff15         1736
6     sales_lag15         1392
0       store_nbr          878
2     onpromotion          684
4   dia_da_semana          483
5      mes_d

##### Gera as previsões e faz a análise delas

In [90]:
# Gera as previsões do modelo
print("Manipulando dados para as previsões...")
dados_teste = temp[temp["id"] >= 3000888].copy()
dados_teste = dados_teste.drop(var_a_remover, axis = 1)

print("Gerando as previsões...")
y_final_pre = reg_lgbm.predict(dados_teste)

# Retorno dos valores normalizados, se for o caso
if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
    print("Dados originais normalizados por Z-Score, retornando valores à normalidade.")
#    y_final = zscore_scaler.inverse_transform(y_final_pre.reshape(-1, 1))
    y_final_pre = pd.DataFrame(y_final_pre)
    y_final_pre["a"] = 0
    y_final_pre["b"] = 0
    y_final_pre["c"] = 0
    y_final = pd.DataFrame(zscore_scaler.inverse_transform(y_final_pre))
    y_final.drop([1, 2], axis = 1, inplace = True)
    y_final = y_final.to_numpy(y_final[0])
elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
    y_final = y_final_pre.copy()
else:
    print("Dados originais normalizados por Log, restituindo à normalidade.")
    y_final = np.expm1(y_final_pre)

y_final_lgbm = y_final.copy()

print("\nVerificando previsões de valores de venda negativos...")
if len(y_final_lgbm[y_final_lgbm < 0]) > 0:
    print("Convertendo valores negativos de venda em zero...")
    # Ajuste das previsões negativas, transformando o valor negativo para zero
    negativas = len(y_final_lgbm[y_final_lgbm < 0])
    y_final_lgbm[y_final_lgbm < 0] = 0
else:
    negativas = 0
    print("Sem previsões de vendas com valores negativos.")
print("\nPrevisões geradas com sucesso. Procedendo com a análise.\n")

vendas = round((y_final_lgbm.sum() / temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum())*100, 2)
perda = 100 - vendas

print("*****")
print("Percentual de preenchimento de vendas:", vendas,"% dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)")
print("Total estimado de vendas futuras foi de", round(y_final_lgbm.sum(), 2))
print("Valor de venda acumulado dos últimos 15 dias do dataset de treino:", round(temp_snorm[(temp_snorm["date"] <= "2017-08-15") & (temp_snorm["date"] >= "2017-08-01")]["sales"].sum(), 2))
if perda < 0:
    print("Isso é o equivalente a um ganho teórico de",abs(perda), "% de faturamento.\n")
else: print("Isso é o equivalente a uma perda teórica de",round(perda, 2), "% de faturamento.")
print("*****")

# Análise das previsões
print("Valor mínimo das vendas previstas: " + str(pd.DataFrame(y_final_lgbm).min().values))
print("Valor da mediana das vendas previstas: " + str(pd.DataFrame(y_final_lgbm).median().values))
print("Valor máximo das vendas previstas: " + str(pd.DataFrame(y_final_lgbm).max().values))
if negativas == 0:
    print("Sem registro de previsões negativas através deste modelo")
else:
    print("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_lgbm))*100, 2)) + "% do total)")
print("\nRelação das frequências dos valores previstos:")
print("     -=+ valores mais frequentes +=-")
print(pd.DataFrame(y_final_lgbm).value_counts().head(10))
print("     -=+ valores com menor frequência +=-")
print(pd.DataFrame(y_final_lgbm).value_counts().tail(10))

Manipulando dados para as previsões...
Gerando as previsões...

Verificando previsões de valores de venda negativos...
Convertendo valores negativos de venda em zero...

Previsões geradas com sucesso. Procedendo com a análise.

*****
Percentual de preenchimento de vendas: 8.72 % dos últimos 15 dias do dataset de treino (01/08/2017 a 15/08/2017)
Total estimado de vendas futuras foi de 1084668.86
Valor de venda acumulado dos últimos 15 dias do dataset de treino: 12433322.94
Isso é o equivalente a uma perda teórica de 91.28 % de faturamento.
*****
Valor mínimo das vendas previstas: [0.]
Valor da mediana das vendas previstas: [4.206756]
Valor máximo das vendas previstas: [4271.073744]
Quantidade de previsões negativas (tranformadas para zero): 304 (1.07% do total)

Relação das frequências dos valores previstos:
     -=+ valores mais frequentes +=-
0       
1.678682    1227
2.385674     765
1.708007     578
2.195347     330
1.694635     313
0.000000     304
1.677181     214
1.896298     181

##### Registro em texto e gera arquivo de submissão

In [91]:
with open("registros_lgbm.txt", "a") as registros:
    registros.write("################ INÍCIO DE REGISTRO - MODELO LGBM " + nome_modelo + " ################\n")
    registros.write("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".")
    registros.write("\nModelo utilizado **não salvo**!")
    if temp[temp.index == "2017-08-15"]["gasolina"].unique() < 0:
        registros.write("\nDados com normalização por ***Z-SCORE***")
    elif temp[temp.index == "2017-08-15"]["gasolina"].unique() > 5:
        registros.write("\nDados ***sem normalização***")
    else:
        registros.write("\nDados com normalização por ***LOG***")
    registros.write("\nPeríodo de análise entre " + str(dados_treino.index.min().strftime("%d-%m-%Y")) + " e " + str(dados_treino.index.max().strftime("%d-%m-%Y")) + ". \n")
    if temp[temp["store_nbr"] == 9].loc["2017-04-02"]["sales"].max() == 38422.625:
        registros.write("Atenção: análise realizada com outliers presentes nas variáveis!\n")
    else:
        registros.write("Análise realizada sem presença de outliers nas variáveis.\n")
    registros.write("\nScores obtidos no controle: " + str(scores_controle))
    if "negativo" in scores_controle:
        registros.write("\nControle possui score(s) negativo(s), sem média.")
        registros.write("\nControle possui score(s) negativo(s), sem mediana.")
        registros.write("\nControle possui score(s) negativo(s), sem registro de melhor score.\n")
    else:
        registros.write("\nMédia dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).mean().values))
        registros.write("\nMediana dos scores obtidos no controle:" + str(pd.DataFrame(scores_controle).median().values))
        registros.write("\nMelhor score obtido no controle:" + str(pd.DataFrame(scores_controle).min().values))
    registros.write("\nCoeficiente de determinação R2 do modelo:"+ str(round(coef_lgbm, 2)) + "\n")

    
    registros.write("\nFeatures utilizadas no modelo e seus respectivos impactos:\n")
    importancias = pd.DataFrame(reg_lgbm.feature_importances_, reg_lgbm.feature_name_)
    importancias.reset_index(inplace = True)
    importancias.columns = ["variável", "importância"]
    importancias = importancias.sort_values(by = "importância", ascending = False)
    registros.write(importancias.to_string(index = False) + "\n")
    if discard_index.sum() > 0:
        registros.write("\nFeatures sem importância: "+ str(dados_treino.drop(var_a_remover, axis = 1).columns.values[discard_index]) + "\n")
    else: registros.write("\nNão foram identificadas features sem importância. \n")
    registros.write("\nDataset timeseries split em " + str(tsgen.n_splits) + " subconjuntos.")
    registros.write("\nTamanho de teste para cada split: " + str(tsgen.test_size) + " observações.\n")
    registros.write("\nParâmetros do modelo:\n")
    registros.write(str(reg_lgbm.get_params()))
    registros.write("\n")
    registros.write("\n ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO MODELO +++===---\n")
    registros.write("\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_lgbm))*100, 2)) + "% do total).\n")

    registros.write("\nRelação das frequências dos valores previstos:\n")
    registros.write("     -=+ valores mais frequentes +=-\n")
    maisfreq = pd.DataFrame(y_final_lgbm).value_counts().head(10)
    registros.write(maisfreq.to_string() + "\n")
    registros.write("     -=+ valores com menor frequência +=-\n")
    menosfreq = (pd.DataFrame(y_final_lgbm).value_counts().tail(10))
    registros.write(menosfreq.to_string() + "\n")
    registros.write("\n################ FINAL DE REGISTRO - MODELO LGBM " + nome_modelo + " ################\n")
    registros.write("\n")
    registros.write("\n")

ext_final = (".csv")
save_final = ("resultado_lgbm_"+(nome_modelo + ext_final))
submissao = pd.read_csv("./csv/sample_submission.csv")
submissao["sales"] = y_final_lgbm
resultado = submissao[["id", "sales"]]
resultado.to_csv("./resultados/"+save_final, index=False)

#### Modelo ensemble

In [171]:
print("Verificando as modelagens já realizadas:")
try:
    print("  Verificando LightGBM...")
    y_final_lgbm
except NameError:
    print("     Modelagem LGBM não realizada, favor providenciar.")
else: print("     Modelo LightGBM encontrado, prosseguindo.")

try:
    print("  Verificando DecisionTree...")
    y_final_dt
except NameError:
    print("     Modelagem DT não realizada, favor providenciar.")
else: print("     Modelo DecisionTree encontrado, prosseguindo.")

try:
    print("  Verificando XGBoost...")
    y_final_xgb
except NameError:
    print("     Modelagem XGB não realizada, favor providenciar.")
else:
    print("     Modelo XGBoost encontrado, prosseguindo.")
    print("\nTodos os modelos necessários para o ensemble foramm encontrados, realizando a união.")
    
nome_modelo = datetime.now().strftime("%Y%m%d-%H%M")
ext_final = (".csv")
save_final = ("resultado_ensemble_"+(nome_modelo + ext_final))
submissao = pd.read_csv("./csv/sample_submission.csv")
submissao["sales"] = (y_final_lgbm+y_final_dt+y_final_xgb)/3
resultado = submissao[["id", "sales"]]
resultado.to_csv("./resultados/"+save_final, index=False)
print("\nEnsemble realizado com sucesso. Arquivo criado:", save_final, ".")
print("Realizando registro em arquivo de texto dos hiperparâmetros das modelagens individuais...")

# Realiza o registro em arquivo de texto
with open("registros_ensamble.txt", "a") as registros:
    registros.write("################ INÍCIO DE REGISTRO - MODELO Ensamble " + nome_modelo + " ################\n")
    
    registros.write("\nRegistro gerado em " + datetime.now().strftime("%d-%m-%Y") + " às " + datetime.now().strftime("%H:%M") + ".\n")
    
    registros.write("\n     .,;-=+++=-;,. .,;-=+++=-;,. ***Base XGBoost*** .,;-=+++=-;,. .,;-=+++=-;,. \n")

    registros.write("\nParâmetros do *Modelo XGBoost*:\n")
    registros.write("           base_score: " + str(reg_xgb.base_score) + "\n")
    registros.write("              booster: " + str(reg_xgb.booster) + "\n")
    registros.write("         n_estimators: " + str(reg_xgb.n_estimators) + "\n")
    registros.write("early_stopping_rounds: " + str(reg_xgb.early_stopping_rounds) + "\n")
    registros.write("            objective: " + reg_xgb.objective+ "\n")
    registros.write("            max_depth: " + str(reg_xgb.max_depth) + "\n")
    registros.write("        learning_rate: " + str(reg_xgb.learning_rate) + "\n")
    registros.write("   enable_categorical: " + str(reg_xgb.enable_categorical) + "\n")
    registros.write("               device: " + reg_xgb.device + "\n")
    registros.write("  validade_parameters: True" + "\n")
    registros.write("\n")
    registros.write(" ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO *Modelo XGBoost* +++===---\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_xgb).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_xgb).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_xgb).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final))*100, 2)) + "% do total).\n\n\n")


    registros.write("\n     .,;-=+++=-;,. .,;-=+++=-;,. ***Base LightGBM*** .,;-=+++=-;,. .,;-=+++=-;,. \n")

    registros.write("\nCoeficiente de determinação R2 do *Modelo LightGBM*:"+ str(round(coef_lgbm, 2)) + "\n")

    registros.write("\nParâmetros do *Modelo LightGBM*:\n")
    registros.write(str(reg_lgbm.get_params()))
    registros.write("\n")
    registros.write("\n ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO MODELO +++===---\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_lgbm).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_lgbm))*100, 2)) + "% do total).\n")


    registros.write("\n     .,;-=+++=-;,. .,;-=+++=-;,. ***Base DecisionTree*** .,;-=+++=-;,. .,;-=+++=-;,. \n")

    registros.write("\nCoeficiente de determinação R2 do *Modelo DecisionTree*:"+ str(round(coef_dt, 2)) + "\n")

    registros.write("\nParâmetros do *Modelo DecisionTree*:\n")
    registros.write(str(reg_dt.get_params()))
    registros.write("\n")
    registros.write("\n ---===+++ ANÁLISE DAS PREVISÕES REALIZADAS PELO MODELO +++===---\n")
    registros.write("Valor mínimo das vendas previstas:" + str(pd.DataFrame(y_final_dt).min().values) + "\n")
    registros.write("Valor da mediana das vendas previstas:" + str(pd.DataFrame(y_final_dt).median().values) + "\n")
    registros.write("Valor máximo das vendas previstas:" + str(pd.DataFrame(y_final_dt).max().values) + "\n")
    if negativas == 0:
        registros.write("Sem registro de previsões negativas através deste modelo.\n")
    else:
        registros.write("Quantidade de previsões negativas (tranformadas para zero): " + str(negativas) + " (" + str(round((negativas/len(y_final_dt))*100, 2)) + "% do total).\n")

    registros.write("\n")
    registros.write("################ FIM DE REGISTRO - MODELO Ensamble " + nome_modelo + " ################\n")
    registros.write("\n")
    registros.write("\n")

print("Pronto.")

Verificando as modelagens já realizadas:
  Verificando LightGBM...
     Modelo LightGBM encontrado, prosseguindo.
  Verificando DecisionTree...
     Modelo DecisionTree encontrado, prosseguindo.
  Verificando XGBoost...
     Modelo XGBoost encontrado, prosseguindo.

Todos os modelos necessários para o ensemble foramm encontrados, realizando a união.

Ensemble realizado com sucesso. Arquivo criado: resultado_ensemble_20240626-1408.csv .
